In [1]:
from math import floor
from pystac_client import Client
import planetary_computer as pc
import numpy as np
import rasterio
import requests, pyproj
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
from rasterio.merge import merge
import rasterio.mask
from rasterio.features import shapes
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import transform, unary_union
import networkx as nx

In [78]:
CONNECTIVITY_DISTANCE = 100    # Max distance two patches can be apart and be considered connected (meter)
MIN_PATCHSIZE = 1000    # Min patch size to be included in analysis (sq meter)

In [67]:
# Get distict boundaries as geojson

URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM2/boundary-CRI-San_Jose-ADM2.geojson'
Districts_json = requests.get(URL).json()

In [4]:
# Convert district-boundary geojsons to Shapely polygons

def geojson_to_polygons(g):
    result = []
    for feature in g['features']:
        name = feature['properties']['shapeName']
        coordpairs = [(float(i[0]), float(i[1])) for i in feature['geometry']['coordinates'][0][0]]
        result.append((name, Polygon(coordpairs)))
    return result

district_polys = geojson_to_polygons(Districts_json)

In [5]:
# Get bounding box and target EPSG for union of district polygons

all_districts = MultiPolygon([i[1] for i in district_polys])
bounds = all_districts.bounds

centroid_lon = all_districts.centroid.xy[0][0]
centroid_lat = all_districts.centroid.xy[1][0]
target_epsg = (32600 + [0, 100][int(centroid_lat < 0)]) + floor((180 + centroid_lon) / 6) + 1
# EPSG is 32600 (or 32700 if lat is neg) + longitude zone. Each zone is six degrees, and first zone is 1.

In [6]:
# Get ESA WorldCover raster tiles covering AOI

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["esa-worldcover"],
    bbox=bounds,
)

items = list(search.get_items())

rasters_to_mosaic = []
signed_hrefs = [pc.sign(i.assets["map"].href) for i in items]
for href in signed_hrefs:
    raster = rasterio.open(href)
    rasters_to_mosaic.append(raster)

In [7]:
# Stitch rasters together

mosaic, mosaic_transform = merge(rasters_to_mosaic)

In [8]:
# This function clips and masks raster
# Adapted from https://gis.stackexchange.com/a/387772

def mask_raster_with_geometry(raster, transform, shapes, **kwargs):
    """Wrapper for rasterio.mask.mask to allow for in-memory processing.

    Docs: https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html

    Args:
        raster (numpy.ndarray): raster to be masked with dim: [H, W]
        transform (affine.Affine): the transform of the raster
        shapes, **kwargs: passed to rasterio.mask.mask

    Returns:
        masked: numpy.ndarray or numpy.ma.MaskedArray with dim: [H, W], and new affine transform
    """
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(
            driver='GTiff',
            height=raster.shape[0],
            width=raster.shape[1],
            count=1,
            dtype=raster.dtype,
            transform=transform,
        ) as dataset:
            dataset.write(raster, 1)
        with memfile.open() as dataset:
            output, new_transform = rasterio.mask.mask(dataset, shapes, **kwargs)
    return output.squeeze(0), new_transform

In [72]:
# Reclassify from ESA WorldCover classes to habitat/nonhabitat

def classify_habitat(r):  # Note: order is important
    r[r == 60] = 1    # sparse veg
    r[r >= 90] = 1    # herbaceous wetland, mangrove, lichen & moss
    r[r == 80] = 0    # permanent open water
    r[r == 70] = 0    # snow/ice
    r[r == 40] = 0    # cropland
    r[r == 50] = 0    # built up
    r[r >= 10] = 1    # tree, shrub, grassland
    r[r == 0] = 0

    return r

In [10]:
# This transforms fro EPSG:4326 to target EPSG

project = pyproj.Transformer.from_crs(
    pyproj.CRS.from_epsg(4326), # source coordinate system
    pyproj.CRS.from_epsg(target_epsg), # destination coordinate system
    always_xy=True
)

In [79]:
connectivity_indicator = {}
for district_data in district_polys:  # Loop over districts
    district_name = district_data[0]
    district_geom = district_data[1]

    # Clip raster to district boundary
    clipped_raster, clip_transform = mask_raster_with_geometry(mosaic[0], mosaic_transform, [district_geom], crop=True)
    
    # Classify clipped raster as habitat/nonhabitat
    hab_raster = classify_habitat(clipped_raster)
    
    # Vectorize and collect only the habitat patches (as opposed to nonhabitat)
    with rasterio.Env():
        image = hab_raster
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, transform=clip_transform)) if v == 1)
    shapelist = list(results)
    shapelist = shapelist[:-1]
    all_patches = {  # project to UTM so that spatial unit is meter
        i: transform(project.transform, Polygon(shapelist[i]['geometry']['coordinates'][0])) for i in range(len(shapelist))
    }
    patches = {
        i: all_patches[i] for i in all_patches if all_patches[i].area >= MIN_PATCHSIZE # Remove very small patches from consideration
    }
    within_distance = []
    for i in patches:
        for j in patches:
            if j > i:
                if patches[i].distance(patches[j]) <= CONNECTIVITY_DISTANCE:  # Collect pairs of patches within connectivity distance
                    within_distance.append((i, j))

    # Find groups of connected patches
    G = nx.Graph()
    G.add_nodes_from(list(patches.keys()))
    G.add_edges_from(within_distance)
    connected_patch_clusters = nx.connected_components(G)
    
    total_area = sum([patches[i].area for i in patches])
    cluster_areas = []
    for i in connected_patch_clusters:
        cluster_areas.append(sum([patches[j].area for j in i]))
    connectivity_indicator[district_name] = sum([i**2 for i in cluster_areas]) / (total_area**2)
    

In [12]:
connectivity_indicator

{'San Jose': 0.6972844454195042,
 'Alajuela': 0.2509144471439529,
 'Moravia': 0.9629849521330002,
 'Paraíso': 0.255951095413317,
 'Poás': 0.16702459080916154,
 'Mora': 0.316104901575237,
 'Alvarado': 0.06389461119000793,
 'Oreamuno': 0.9999001856566834,
 'Heredia Urban': 0.37237589803365284,
 'Heredia Rural': 0.47466346454040775,
 'Tibás': 0.57984084174516,
 'Vasquez de Coronado': 0.6381071001000168,
 'Atenas': 1.0000000000000002,
 'Desamparados': 0.46950708706251654,
 'Aserrí': 0.4793307889029919,
 'Santo Domingo': 0.09732983839538048,
 'El Guarco': 0.36731615179082067,
 'Montes de Oca': 0.9671781759563389,
 'Goicoechea': 0.9413713968859365,
 'Cartago': 0.9604917842558767,
 'San Pablo': 0.9899425418594883,
 'Curridabat': 0.9554204487999002,
 'Alajuelita': 0.41052804625864964,
 'Barva': 0.9996343791279675,
 'Belén': 0.9478446535411826,
 'Escazú': 0.6328730031801533,
 'Flores': 0.959067458038766,
 'La Unión': 0.5419508817126665,
 'San Isidro': 0.1618281707744469,
 'San Rafael': 0.999200

In [13]:
# Write outputs to CSV

ofile = open('connectivity_outputs.csv', 'w')
ofile.write('District,Connectivity\n')
for dist in connectivity_indicator:
    ofile.write('{0},{1}\n'.format(dist, str(connectivity_indicator[dist])))
ofile.close()

In [81]:
'''
from shapely.geometry import mapping, Polygon
import fiona

# Define a polygon feature geometry with one attribute
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

# Write a new Shapefile
with fiona.open('filename.shp', 'w', 'ESRI Shapefile', schema) as c:
    ## If there are multiple geometries, put the "for" loop here
    count = 1
    for i in patches:   # Will do patches dict for most recent district
        c.write({
            'geometry': mapping(patches[i]),
            'properties': {'id': count},
        })
        count += 1
'''